In [1]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py
!pip install cma
!pip install nevergrad
!pip install stable_baselines3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-dev set to manually installed.
software-properties-common is already the newest version (0.96.24.32.18).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  glew-utils
The following NEW packages will be installed:
  libgl1-mesa-glx libglew-dev libglew2.0 libosmesa6 libosmesa6-dev
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,916 kB of archives.
After this operation, 12.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgl1-mesa-glx amd64 20.0.8-0ubuntu1~18.04.1 [5,532 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libglew2.0 amd64 2.0.0-5 [140 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/univ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 15.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 420 kB 14.4 MB/s 
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=de6227fa9ab005c49984f724656cfbbd07171fd37b96c81bb043d2e8cfcaf044
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 13.9 MB/s 
     |████████████████████████████████| 1.5 MB 54.7 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616826 sha256=e5ba7564be5d047bfa7f5360ba9f721c9bdfa9b6da70f2f2f7400b7e34f3d

In [2]:
!unzip classes.zip

Archive:  classes.zip
  inflating: classes/actor_critic.py  
  inflating: classes/data.npy        
  inflating: classes/dropo.py        
   creating: classes/env/
 extracting: classes/env/__init__.py  
   creating: classes/env/__pycache__/
  inflating: classes/env/__pycache__/__init__.cpython-38.pyc  
  inflating: classes/env/__pycache__/custom_hopper.cpython-38.pyc  
  inflating: classes/env/__pycache__/jinja_mujoco_env.cpython-38.pyc  
  inflating: classes/env/__pycache__/mujoco_env.cpython-38.pyc  
  inflating: classes/env/__pycache__/random_mass_hopper.cpython-38.pyc  
  inflating: classes/env/__pycache__/template_renderer.cpython-38.pyc  
   creating: classes/env/assets/
  inflating: classes/env/assets/hopper.xml  
  inflating: classes/env/custom_hopper.py  
  inflating: classes/env/mujoco_env.py  
  inflating: classes/vanilla.py      


In [3]:
from classes.dropo import *
from classes.env.custom_hopper import *
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

Compiling /usr/local/lib/python3.7/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.7/dist-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr/local
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr/local/lib
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuexten

In [5]:
env = gym.make('CustomHopper-target-v0')
K = 100
lambda_t = 50
epsilon = 1e-3
instances = 2
dropo = Dropo(env, lambda_t, epsilon, K, instances)
budget = 1
res = dropo.optimize(budget)

In [6]:
n_episodes = 100000
device = 'cpu'
env_source = gym.make('CustomHopper-source-v0')
env_source.set_dropo_parameters(res)
env_source.set_distribution('truncnormal')

In [7]:
"""
  Training
"""
model = PPO("MlpPolicy", env_source, learning_rate=1e-3, gamma=0.99, verbose = 0, device = device)
model.learn(total_timesteps=n_episodes, n_eval_episodes = 50, eval_log_path = '/content/ppo')
model.save("ppo_hopper_dropo.mdl")

In [8]:
"""
  Testing
"""
#env = gym.make('CustomHopper-source-v0')
env = gym.make('CustomHopper-target-v0')
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=50)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


mean_reward:965.72 +/- 13.87
